# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7efeb0815160>
├── 'a' --> tensor([[-0.9350,  0.7279,  1.4273],
│                   [ 0.4053,  1.6148, -0.6397]])
└── 'x' --> <FastTreeValue 0x7efeb087ac70>
    └── 'c' --> tensor([[-0.4748, -1.2389, -2.2595, -0.0433],
                        [-0.8744,  0.0724, -0.8893, -0.9001],
                        [ 1.1261, -0.7622, -0.2621,  0.1067]])

In [4]:
t.a

tensor([[-0.9350,  0.7279,  1.4273],
        [ 0.4053,  1.6148, -0.6397]])

In [5]:
%timeit t.a

70.7 ns ± 1.49 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7efeb0815160>
├── 'a' --> tensor([[ 1.6539, -0.4743, -1.1962],
│                   [-0.6364,  1.4881, -0.6925]])
└── 'x' --> <FastTreeValue 0x7efeb087ac70>
    └── 'c' --> tensor([[-0.4748, -1.2389, -2.2595, -0.0433],
                        [-0.8744,  0.0724, -0.8893, -0.9001],
                        [ 1.1261, -0.7622, -0.2621,  0.1067]])

In [7]:
%timeit t.a = new_value

78.9 ns ± 0.973 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9350,  0.7279,  1.4273],
               [ 0.4053,  1.6148, -0.6397]]),
    x: Batch(
           c: tensor([[-0.4748, -1.2389, -2.2595, -0.0433],
                      [-0.8744,  0.0724, -0.8893, -0.9001],
                      [ 1.1261, -0.7622, -0.2621,  0.1067]]),
       ),
)

In [10]:
b.a

tensor([[-0.9350,  0.7279,  1.4273],
        [ 0.4053,  1.6148, -0.6397]])

In [11]:
%timeit b.a

66 ns ± 1.29 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0668, -0.6080,  0.4004],
               [ 0.1907,  0.0284, -0.1178]]),
    x: Batch(
           c: tensor([[-0.4748, -1.2389, -2.2595, -0.0433],
                      [-0.8744,  0.0724, -0.8893, -0.9001],
                      [ 1.1261, -0.7622, -0.2621,  0.1067]]),
       ),
)

In [13]:
%timeit b.a = new_value

615 ns ± 7.58 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.01 µs ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.3 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

283 µs ± 4.96 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

267 µs ± 6.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7efdf8ab0dc0>
├── 'a' --> tensor([[[-0.9350,  0.7279,  1.4273],
│                    [ 0.4053,  1.6148, -0.6397]],
│           
│                   [[-0.9350,  0.7279,  1.4273],
│                    [ 0.4053,  1.6148, -0.6397]],
│           
│                   [[-0.9350,  0.7279,  1.4273],
│                    [ 0.4053,  1.6148, -0.6397]],
│           
│                   [[-0.9350,  0.7279,  1.4273],
│                    [ 0.4053,  1.6148, -0.6397]],
│           
│                   [[-0.9350,  0.7279,  1.4273],
│                    [ 0.4053,  1.6148, -0.6397]],
│           
│                   [[-0.9350,  0.7279,  1.4273],
│                    [ 0.4053,  1.6148, -0.6397]],
│           
│                   [[-0.9350,  0.7279,  1.4273],
│                    [ 0.4053,  1.6148, -0.6397]],
│           
│                   [[-0.9350,  0.7279,  1.4273],
│                    [ 0.4053,  1.6148, -0.6397]]])
└── 'x' --> <FastTreeValue 0x7efdf8ab0d90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.4 µs ± 921 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7efeb0822bb0>
├── 'a' --> tensor([[-0.9350,  0.7279,  1.4273],
│                   [ 0.4053,  1.6148, -0.6397],
│                   [-0.9350,  0.7279,  1.4273],
│                   [ 0.4053,  1.6148, -0.6397],
│                   [-0.9350,  0.7279,  1.4273],
│                   [ 0.4053,  1.6148, -0.6397],
│                   [-0.9350,  0.7279,  1.4273],
│                   [ 0.4053,  1.6148, -0.6397],
│                   [-0.9350,  0.7279,  1.4273],
│                   [ 0.4053,  1.6148, -0.6397],
│                   [-0.9350,  0.7279,  1.4273],
│                   [ 0.4053,  1.6148, -0.6397],
│                   [-0.9350,  0.7279,  1.4273],
│                   [ 0.4053,  1.6148, -0.6397],
│                   [-0.9350,  0.7279,  1.4273],
│                   [ 0.4053,  1.6148, -0.6397]])
└── 'x' --> <FastTreeValue 0x7efdf90f6ca0>
    └── 'c' --> tensor([[-0.4748, -1.2389, -2.2595, -0.0433],
                        [-0.8744,  0.0724, -0.8893, -0.9001],
                 

In [23]:
%timeit t_cat(trees)

40.6 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78.6 µs ± 765 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.4748, -1.2389, -2.2595, -0.0433],
                       [-0.8744,  0.0724, -0.8893, -0.9001],
                       [ 1.1261, -0.7622, -0.2621,  0.1067]],
              
                      [[-0.4748, -1.2389, -2.2595, -0.0433],
                       [-0.8744,  0.0724, -0.8893, -0.9001],
                       [ 1.1261, -0.7622, -0.2621,  0.1067]],
              
                      [[-0.4748, -1.2389, -2.2595, -0.0433],
                       [-0.8744,  0.0724, -0.8893, -0.9001],
                       [ 1.1261, -0.7622, -0.2621,  0.1067]],
              
                      [[-0.4748, -1.2389, -2.2595, -0.0433],
                       [-0.8744,  0.0724, -0.8893, -0.9001],
                       [ 1.1261, -0.7622, -0.2621,  0.1067]],
              
                      [[-0.4748, -1.2389, -2.2595, -0.0433],
                       [-0.8744,  0.0724, -0.8893, -0.9001],
                       [ 1.1261, -0.7622, -0.2621,  0.1067]],

In [26]:
%timeit Batch.stack(batches)

103 µs ± 1.82 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.4748, -1.2389, -2.2595, -0.0433],
                      [-0.8744,  0.0724, -0.8893, -0.9001],
                      [ 1.1261, -0.7622, -0.2621,  0.1067],
                      [-0.4748, -1.2389, -2.2595, -0.0433],
                      [-0.8744,  0.0724, -0.8893, -0.9001],
                      [ 1.1261, -0.7622, -0.2621,  0.1067],
                      [-0.4748, -1.2389, -2.2595, -0.0433],
                      [-0.8744,  0.0724, -0.8893, -0.9001],
                      [ 1.1261, -0.7622, -0.2621,  0.1067],
                      [-0.4748, -1.2389, -2.2595, -0.0433],
                      [-0.8744,  0.0724, -0.8893, -0.9001],
                      [ 1.1261, -0.7622, -0.2621,  0.1067],
                      [-0.4748, -1.2389, -2.2595, -0.0433],
                      [-0.8744,  0.0724, -0.8893, -0.9001],
                      [ 1.1261, -0.7622, -0.2621,  0.1067],
                      [-0.4748, -1.2389, -2.2595, -0.0433],
                   

In [28]:
%timeit Batch.cat(batches)

226 µs ± 5.43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

598 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
